In [1]:
from Point_Er import Point
from Lattice_Er import Lattice
from Simulator_Er import Simulator
from utils import *
from EnergyTransfer_Er import *
import numpy as np

In [2]:
tag_default={'c0':1e-39, # Yb-Yb resonant energy transfer
        'Ws': 639,
 'E1E0': 88.12753083306136,
 'E2E1': 13.728308752002313,
 'E2E0': 105.00663885847584,
 
 'E3E2': 0.6904748414556272,
 'E3E1': 40.06626483314129,
 'E3E0': 107.07825106403719,

 'E4E3': 1.4142534182563467,
 'E4E2': 49.124834957391506,
 'E4E1': 45.114305779338295,
 'E4E0': 1009.6221517188111,

 'E5E4': 0.5491077883920105 + 3909/3.5,
 'E5E3': 46.481404188403104,
 'E5E2': 28.889483458690968,
 'E5E1': 378.15231194559027,
 'E5E0': 919.1044353717751,

 'E6E5': 0.02617036285192619 + 14290358.698538769/3.5,
 'E6E4': 8.841624545216064,
 'E6E3': 47.60084543949401,
 'E6E2': 41.09061870168263,
 'E6E1': 71.35702052573745,
 'E6E0': 2812.803587953125,

 'E7E6': 0.4092613138830535 + 2927313.4554100684/3.5,
 'E7E5': 0.01904121955274265,
 'E7E4': 3.4467583618029134,
 'E7E3': 93.44157758618482,
 'E7E2': 162.98778196545229,
 'E7E1': 334.1120016219258,
 'E7E0': 2256.2758284193}

# Progress file

In [3]:
import os
import pickle
from datetime import datetime
from time import sleep


# load or initialize progress
def load_or_initialize_progress(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'rb') as f:
            return pickle.load(f)
    else:
        return {}

# save progress
def save_progress(file_path, data):
    with open(file_path, 'wb') as f:
        pickle.dump(data, f)

progress_folder = 'Yuxuan_data_files'
if not os.path.exists(progress_folder):
    os.makedirs(progress_folder)

# unique filename
current_date = datetime.now().strftime('%m_%d_%Y')
base_filename = f'Progress_{current_date}'
index = 1
progress_filepath = os.path.join(progress_folder, f'{base_filename}_{index}.ipynb')
    
while os.path.exists(progress_filepath):
    index += 1
    progress_filepath = os.path.join(progress_folder, f'{base_filename}_{index}.ipynb')


# The first time to run the code, the folder 'Chuanyu_data_files' will have a new file: 'myC_progress.pkl'
# if there is no at least one combination finished, there would be no 'myC_progress.pkl' in the directory
# check the progress data if the program is interruptted


"""
file_path = 'Chuanyu_data_files/myC_progress.pkl'

with open(file_path, 'rb') as file:
    progress_data = pickle.load(file)

for key, value in progress_data.items():
    print(key, value)
"""

# Load or initialize progress data
# if progress_file_path doesn't exist, then return a {}
# if progress_file_path exists, then return a progress_data, which is an incomplete myC
# after interruption, run code and the the progress bar will start from the next combination
# after finish, the progress file is large: ~60M

myC = load_or_initialize_progress(progress_filepath)


# Instruction file

In [4]:
import os
import nbformat as nbf
from datetime import datetime

def create_instruction_notebook(guide_folder='Yuxuan_data_files', saturation_plot_file='SaturationCurves.py', optimal_percentage_file='OptimalPercentage.py', population_evolution_file='PopulationEvolution.py'):
    """
    Create a Jupyter Notebook with instructions and code to load data and generate plots.
    
    Parameters:
    - guide_folder (str): Folder where the guide notebook will be saved.
    - saturation_plot_file (str): Path to the Python file containing the SaturationPlot class.
    - optimal_percentage_file (str): Path to the Python file containing the SinglePowerDensityPlot class.
    - population_evolution_file (str): Path to the Python file containing the PopulationEvolutionPlot class.
    """

    if not os.path.exists(guide_folder):
        os.makedirs(guide_folder)

    # unique filename
    current_date = datetime.now().strftime('%m_%d_%Y')
    base_filename = f'Guide_{current_date}'
    index = 1
    notebook_path = os.path.join(guide_folder, f'{base_filename}_{index}.ipynb')
    while os.path.exists(notebook_path):
        index += 1
        notebook_path = os.path.join(guide_folder, f'{base_filename}_{index}.ipynb')

    # quick notes
    instructions = input("Please provide instructions for this dataset: ")

    # create a new ipynb object
    nb = nbf.v4.new_notebook()

    # add a markdown cell with the provided instructions
    nb.cells.append(nbf.v4.new_markdown_cell(f"# Guide for Dataset - {current_date}_{index}\n\n{instructions}"))

    # add a code cell for data loading
    nb.cells.append(nbf.v4.new_code_cell(f"""\
import pickle
import os


# Load the data

filepath = f'myC_{current_date}_{index}.pkl'
with open(filepath, 'rb') as f:
    data = pickle.load(f)

print("Data loaded successfully")
"""))
    



    # Create the first new code cell for checking percentages and power densities
    new_cell_1 = """
# check the percentages and power density

percentages = sorted(data.keys())
power_densities = sorted({k for subdict in data.values() for k in subdict.keys()})
print('Percentages:', percentages)
print('Power Densities:', sorted(power_densities))
    """

    # Append the first new cell to the notebook
    nb.cells.append(nbf.v4.new_code_cell(new_cell_1))

    # Create the second new code cell for calculating the ratio
    new_cell_2 = """\
ratio = {}
for p in power_densities:
    ratio[(0.02, p)] = (data[0.02][p]['green_avg'], data[0.02][p]['red_avg'], data[0.02][p]['green_red_ratio'])
    ratio[(0.6, p)] = (data[0.6][p]['green_avg'], data[0.6][p]['red_avg'], data[0.6][p]['green_red_ratio'])
ratio
    """

    nb.cells.append(nbf.v4.new_code_cell(new_cell_2))



    # add a code cell for importing and using the SaturationPlot class, SinglePowerDensityPlot class, PopulationEvolutionPlot class
    nb.cells.append(nbf.v4.new_code_cell(f"""\
# Import the SaturationPlot class from the SaturationCurves module
from {os.path.splitext(saturation_plot_file)[0]} import SaturationPlot

# Generate the saturation curves plot
plot = SaturationPlot(data)
plot.generate_plot(output_file='saturation_plot.html')
"""))

    # add 
    nb.cells.append(nbf.v4.new_code_cell(f"""\
# Import the SinglePowerDensityPlot class from the OptimalPercentage module
from {os.path.splitext(optimal_percentage_file)[0]} import SinglePowerDensityPlot


# Provide a list of available power densities
available_power_densities = sorted({{k for subdict in data.values() for k in subdict.keys()}})
print(f'Available power densities: {{available_power_densities}}')

# Prompt the user to select a power density
selected_power_density = float(input(f'Input a power density from the above options: '))

# Generate the single power density plot
single_plot = SinglePowerDensityPlot(data, selected_power_density)
single_plot.generate_plot(output_file='single_power_density_plot.html')
"""))

    # add 
    nb.cells.append(nbf.v4.new_code_cell(f"""\
# Import the PopulationEvolutionPlot class from the PopulationEvolution module
from {os.path.splitext(population_evolution_file)[0]} import PopulationEvolutionPlot

# Provide a list of available percentages

available_percentages = sorted(data.keys())
print(f'Available percentages: {{available_percentages}}')
# Prompt the user to select a percentage
percentage = float(input(f'Input a percentage from the above options: '))

available_power_densities = sorted({{k for k in data[percentage].keys()}})
print(f'Available power densities for percentage {{percentage}}: {{available_power_densities}}')
# Prompt the user to select a power density
power_density = float(input(f'Input a power density from the above options: '))




# Generate the population evolution plot
pop_plot = PopulationEvolutionPlot(data, percentage, power_density)
pop_plot.generate_plot(output_file='population_evolution_plot.html')
"""))

    # Write the notebook to the file
    with open(notebook_path, 'w') as f:
        nbf.write(nb, f)

    print(f"Instruction notebook has been successfully saved to '{notebook_path}'")
    print()


    


In [5]:
create_instruction_notebook('Yuxuan_data_files', 'SaturationCurves_Er.py', 'OptimalPercentage_Er.py', 'PopulationEvolution_Er.py')

Instruction notebook has been successfully saved to 'Yuxuan_data_files\Guide_10_05_2024_1.ipynb'



# Save file

In [6]:
# use pickle module to save the results as a pickle file and reload them later
# The / character cannot be used in file names because it is reserved as a directory separator, replace / with underscores _ or using another safe delimiter.


import pickle
from datetime import datetime
import os
import plotly.graph_objects as go
import nbformat as nbf



class PickleSaver:

    def __init__(self, base_name='Mydata', folder='Yuxuan_data_files'):
     
        self.base_name = base_name
        self.folder = folder


        if not os.path.exists(self.folder):
            os.makedirs(self.folder)

    def save_data(self, data):
      

        current_date = datetime.now().strftime('%m_%d_%Y')
        base_filename = f'{self.base_name}_{current_date}'
        
        index = 1
        filename = os.path.join(self.folder, f'{base_filename}_{index}.pkl')
        while os.path.exists(filename):
            index += 1
            filename = os.path.join(self.folder, f'{base_filename}_{index}.pkl')


        with open(filename, 'wb') as f:
            pickle.dump(data, f)

        print(f"Data has been successfully saved to '{filename}'")
        print()


    def load_all_data(self):
        
        if not os.path.exists(self.folder):
            print(f"Folder '{self.folder}' does not exist.")
            return {}

        files = [f for f in os.listdir(self.folder) if f.endswith('.pkl')]
        all_data = {}

        for file in sorted(files):
            filepath = os.path.join(self.folder, file)
            with open(filepath, 'rb') as f:
                data = pickle.load(f)
                all_data[file] = data

        print(f'Filename = {file}')

        KEY1=[]
        KEY2=[]

        for key1, value1 in data.items():
            KEY1.append(key1)
        print(f'\nAll percentages = {KEY1}')

        for key1, value1 in data.items():
            for key2, value2 in value1.items():
                KEY2.append(key2)
            break
        print(f'\nFor each percentage, all power densities = {KEY2}')

        # use a set to keep track of unique third keys
        unique_third_keys = set()

        print('\nFor each combination of Percentage+Power density, the accessible data: ')
        print()
        for key1, value1 in data.items():
            for key2, value2 in value1.items():
                for key3 in value2.keys():
                    if key3 not in unique_third_keys:
                        unique_third_keys.add(key3)
                        print(f'key3 = {key3}, value3 = {value2[key3]}')

: 

# Execution

Yuxuan_size: 4, 8, 12, 16, 20, 24, 28

Chuanyu_size: 6, 10, 14, 18, 22, 26, 30



In [7]:



# nanoparticle diameter: 8 nm, critical distance: 1 nm
d=22
shell=0
r0=1
defects_fraction = 0.1

# Yb_conc=0.2

Er_conc = [0.02, 0.04, 0.08, 0.16, 0.50]
power_density =  [10, 1*10**2, 1*10**3, 1*10**4, 1*10**5, 1*10**6, 1*10**7]

dt = 10**(-7)
t1 = 40000
t2 = 60000

# Simulate and track progress

for i, c in enumerate(Er_conc):
    if c not in myC:
        myC[c] = {}

    for p in power_density:
        if p not in myC[c]:
            print(f"Running simulation for Er concentration {c}, power density {p}")

            # sigma from MC paper, SI 
            tag_default['laser'] = 0.058 * p
            tag_default['laser_er'] = 0.0084 * p       
            
            my_lattice = Lattice(1-c, c, d, r0, defects_fraction, 0, 0)  # shell

            # my_lattice.introduce_defects()
            
            my_simulator = Simulator(my_lattice, tag=tag_default)
            result = my_simulator.simulate_lifetime(t1, t2)  # r is a dictionary from 'sim_stats'

            # Store result and save progress
            myC[c][p] = result
            # store the current progress to the filepath
            save_progress(progress_filepath, myC)

            # Simulate time delay to represent real simulation time
            sleep(1)

print('\nAll progress have been finished.')

Running simulation for Er concentration 0.02, power density 10


100%|██████████| 20000/20000 [33:15<00:00, 10.02it/s]   


Running simulation for Er concentration 0.02, power density 100


100%|██████████| 20000/20000 [11:16<00:00, 29.58it/s]


Running simulation for Er concentration 0.02, power density 1000


100%|██████████| 20000/20000 [57:13<00:00,  5.83it/s] 


Running simulation for Er concentration 0.02, power density 10000


 16%|█▌        | 6219/40000 [2:39:56<16:34:43,  1.77s/it]

In [8]:
# after all the progress are finished
# The first time to run the code, the folder 'Chuanyu_data_files' will have a new file: 'myC_05_06_2024_1.pkl', which store all data
# The second time to run the code, the folder 'Chuanyu_data_files' will have a new file: 'myC_05_06_2024_2.pkl', which store all data

saver = PickleSaver(base_name='myC', folder='Yuxuan_data_files')
saver.save_data(myC)

Data has been successfully saved to 'Yuxuan_data_files\myC_09_26_2024_2.pkl'



---
### If you want to run the code again, firstly make sure you have run the code above to store the data:

'''
saver = PickleSaver(base_name='myC', folder='Chuanyu_data_files')
saver.save_data(myC)b
all_save_data = saver.load_all_data()
'''
#### In the same directory, there would be a new file: myC_05_08_2024_1, which save all data from this simulation
#### Then you can detele the progress file and to run the code
---